In [39]:
# Importar librerías 
import DataFunctions
import ModelFunctions
import seaborn as sns
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from valuation import Companies, Models
from sklearn.impute import KNNImputer
import statsmodels.api as sm
import numpy as np
import pandas as pd
#from ydata_profiling import ProfileReport

In [40]:
# Cargar Datos
income_statement = DataFunctions.load_full_excel('Data/Income/Income_Statement.xlsx')
balance_statement = DataFunctions.load_full_excel('Data/Balance/Balance_Statement.xlsx')
sp500 = DataFunctions.assets(income_statement=income_statement)

# Si se desea cambiar de periodo de analisis cambiar la siguiente fecha:
prices = yf.download(tickers=sp500, start='2018-09-01', end='2023-09-23', progress=False)['Adj Close'] 

# Filtrado de fechas y correcciones temporales
prices_fiscal = DataFunctions.prices_date(balance_statement=balance_statement, prices=prices, sp500=sp500)

# DataFrame limpio con información consolidada de ambos estados financieros y precios
financial_info = DataFunctions.clean_df(balance_statement=balance_statement, 
                                    income_statement=income_statement, 
                                    sp500=sp500, 
                                    prices_fiscal=prices_fiscal)

# Datos en formato tabular
data_table = DataFunctions.tabular_df(financial_info=financial_info, sp500=sp500)

# Cálculo de ratios financieros 
stock = Companies(data_table)
stock.get_ratios()

# EDA con miras a limpieza de datos
DataFunctions.dqr(stock.ratios)

# Limpieza
stock.clean_ratios()
stock.clean

In [ ]:
import joblib
from locale import setlocale, LC_TIME

database = stock.clean.copy()
loaded_model = joblib.load(open('GBC_bagging_model.pkl', 'rb'))

setlocale(LC_TIME, 'en_US.UTF-8')

# Model
#database.to_csv('data.csv')
data = pd.read_csv('data.csv').drop('Unnamed: 0', axis=1)
data['fiscalDateEnding'] = pd.to_datetime(data['fiscalDateEnding'], format='%Y-%m-%d')
data['Yhat'] = loaded_model.predict(data.drop(['Stock', 'fiscalDateEnding', 'Return'], axis=1))
data = data.sort_values(by = 'fiscalDateEnding', ascending = True)

# Prices
prices = stock.df[['Stock','fiscalDateEnding','Adj Close']]
prices = prices.sort_values(by = 'fiscalDateEnding')

data = data.merge(prices, how = 'left', on = ['fiscalDateEnding','Stock'])
data.head()

,Stock,fiscalDateEnding,PER,PBV,Acid_test,ATR,CCC,ROA,DER,NPM,EM,Return,Yhat,Adj Close
0,GEN,2018-12-31,-397.673958,0.371745,26.970165,0.071512,906.206897,-0.000935,0.862399,-0.013072,1.86327,0,0,9.880098
1,DISH,2018-12-31,18.623365,0.939512,41.532332,0.396907,209.470823,0.050448,0.862399,0.127103,1.86327,0,0,24.969999
2,DLTR,2018-12-31,103.204542,3.398345,26.970165,0.647207,20.407688,0.032928,0.862399,0.050877,1.86327,0,0,90.320000
3,DOV,2018-12-31,134.515580,2.472537,26.970165,0.204184,95.486986,0.018381,0.862399,0.090022,1.86327,0,0,65.714203
4,DOW,2018-12-31,0.000000,0.000000,26.970165,1.740944,8.315140,0.156812,0.862399,0.090073,1.86327,0,0,0.000000


In [ ]:
rf = pd.read_csv("^IRX.csv")
rf.Date = pd.to_datetime(rf['Date'], format='%m/%d/%y')

rf["rf"] = rf["Adj Close"]
rf.drop(["Adj Close"],axis=1 ,inplace=True)

rf["fiscalDateEnding"] = rf.Date
rf.drop(["Date"],axis=1 ,inplace=True)

rf.fillna(method='ffill', inplace=True)

data = data.merge(rf, on="fiscalDateEnding")

investment_rf = {0: 1, 
                 1: 0.8, 
                 2: 0.6, 
                 3: 0.4, 
                 4: 0.2, 
                 5: 0}

initial_capital = 1000000
comision = .00025

#date = '2018-12-31'
date = '2019-06-30'
trade_period = data[data['fiscalDateEnding'] == date]

In [ ]:
import AssetAllocation as AA

In [ ]:
# ASSET PICKER
def pick_assets(data: pd.DataFrame, assets: pd.DataFrame, fiscal_date: str):
    # Previous Stocks Evaluation
    previous_stocks = data[(data['fiscalDateEnding'] == fiscal_date) & (data['Yhat'] == 1)].merge(assets, on= 'Stock', how = 'inner')
    previous_stocks = previous_stocks['Stock'].values
    # Add Missing Stocks
    new_assets = 5 - len(previous_stocks)
    try:
        add_assets = data[(data['fiscalDateEnding'] == fiscal_date) & (data['Yhat'] == 1)].sample(n = new_assets)['Stock'].values
        assets_list = list(previous_stocks) + list(add_assets) 
    except:
        assets_list = []
    return assets_list



In [ ]:
# OMEGA ASSET ALLOCATION
def omegaAA(data: pd.DataFrame, assets: pd.DataFrame, assets_lists: list, fiscal_date: str, mkt_idx: str = '^GSPC'):
    # Omega Optimization
    rf_rate = data.rf.values[0] / 100
    if len(assets_lists) > 0:
        tickers = assets_lists.copy()
        tickers.append(mkt_idx)
        end_date = pd.to_datetime(fiscal_date)
        start_date = end_date + timedelta(days = -365)
        try:
            omega_prices = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
        except:
            try:
                omega_prices = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
            except:
                try:
                    omega_prices = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
                except:
                    omega_prices = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
        omega = AA.asset_allocation(data_stocks=omega_prices[omega_prices.columns[:-1]], data_benchmark=omega_prices[omega_prices.columns[-1]].to_frame(), rf=rf_rate)
        omega_weights = omega.omega(n_port=1)
    else:
        omega_weights = []
    # RF
    print('OW Size:', len(omega_weights))
    rf_percentage = (5 - len(omega_weights)) * .2
    print('RF %:',rf_percentage)
    if rf_percentage > 0:
        assets_lists = assets_lists + ['Rf']
        omega_weights = np.array(omega_weights) * (1 - rf_percentage)
        omega_weights = np.concatenate((omega_weights, [rf_percentage]))
        omega_weights = list(omega_weights).copy()

    # Assets DF
    new_assets = pd.DataFrame([ pd.to_datetime([fiscal_date for i in range(len(omega_weights))]),
                             assets_lists, omega_weights],
                           index=['Date','Stock','W']).transpose()
    assets = pd.concat([assets, new_assets], axis = 0, ignore_index = True)

    # Omega Weights
    omega_weights = {asset:[weight] for asset,weight in zip(assets_lists,omega_weights)}
    omega_weights = pd.DataFrame.from_dict(omega_weights).T.reset_index().rename(columns = {'index':'Stock',0:'Weight'})

    return omega_weights, assets

In [ ]:

def trade(data: pd.DataFrame, assets: pd.DataFrame, operations: pd.DataFrame, positions_status_rf: pd.DataFrame, omega_weights: pd.DataFrame, portfolio_valuation: pd.DataFrame, fiscal_date: str, initial_capital: int = 1000000, commission: float = 0.00025):   
    #omega_weights
    omega_weights = omega_weights[omega_weights['Stock'] != 'Rf']['Weight'].values
    assets_rf = assets[assets['Stock'] == 'Rf'].copy()
    previous_rf_date = assets_rf[assets_rf['Date'] != fiscal_date]['Date'].max()

    try:
        previous_rf_date = previous_rf_date.strftime('%Y-%m-%d')
    except:
        pass

    # Sell Previous Position in Rf
    try:
        bought_rf = operations[(operations['Date'] == previous_rf_date) & (operations['Stock'] == 'Rf') & (operations['Type'] == 'Buy')]
        closed_position = positions_status_rf[
            (positions_status_rf['Rate'] == bought_rf['Price'].values[0]) & (positions_status_rf['Position'] == bought_rf['Position'].values[0])
            ]['Status'].values
    except:
        pass

    if len(bought_rf) > 0 and ('Closed' not in closed_position):
        print('')
        print('Eval: ',len(bought_rf) > 0,' , ','Closed' not in closed_position)
        rf_1 = bought_rf['Price'].values[0]
        rf_position = bought_rf['Position'].values[0]
        income_risk_free =  rf_position * ( 1 + rf_1 *  3/12)
        operations_sell_rf = pd.DataFrame([fiscal_date,'Rf',income_risk_free,rf_1,-income_risk_free,'Sell'], index = ['Date','Stock','X','Price','Position','Type']).T
        close = pd.DataFrame([fiscal_date,rf_1,income_risk_free,'Closed'], index = ['Date','Rate','Position','Status']).T
        positions_status_rf = pd.concat([positions_status_rf,close], axis = 0, ignore_index = True)
        print('Close position')
    else:
         operations_sell_rf = pd.DataFrame(columns = ['Date','Stock','X','Price','Position','Type'])
         income_risk_free = 0
    

    # Capitals Trade
    # Data
    assets = assets[assets['Stock'] != 'Rf'].copy()
    tickers_new = assets[assets['Date'] == fiscal_date]['Stock'].values
    previous_date = assets[assets['Date'] != fiscal_date]['Date'].max()

    try:
        previous_date = previous_date.strftime('%Y-%m-%d')
    except:
        pass

    if len(assets) > 0:
        tickers_previous = assets[assets['Date'] == previous_date]['Stock'].values
        end_date = pd.to_datetime(fiscal_date) + timedelta(days=7)
        tickers = np.unique(np.array(list(tickers_new) + list(tickers_previous)))
        tickers = list(tickers)
        try:
            prices_new = yf.download(tickers=tickers, start = fiscal_date, end = end_date, progress = False)['Adj Close'].iloc[0]
        except:
            try:
                prices_new = yf.download(tickers=tickers, start = fiscal_date, end = end_date, progress = False)['Adj Close'].iloc[0]
            except:
                try:
                    prices_new = yf.download(tickers=tickers, start = fiscal_date, end = end_date, progress = False)['Adj Close'].iloc[0]
                except:
                    prices_new = yf.download(tickers=tickers, start = fiscal_date, end = end_date, progress = False)['Adj Close'].iloc[0]
        prices_new = prices_new.to_frame()
        prices_new.columns = ['Price']
    else:
        prices_new = pd.DataFrame(columns=['Price','Stock']).set_index('Stock')

    # X_T-1
    Xt_prior = operations[(operations['Stock'] != 'Rf') & (operations['Date'] == previous_date)].set_index('Stock')['X'].to_frame()
    Xt_prior = Xt_prior.merge(prices_new, left_index = True, right_index = True, how = 'left')
    Xt_prior['Pos'] = Xt_prior['X'] * Xt_prior['Price']
    Val_Port = Xt_prior['Pos'].sum() # income_risk_free
    print('Capitals:',Xt_prior['Pos'].sum(),' RF: ', income_risk_free)
    if Val_Port == 0:
        Val_Port = initial_capital
    Xt_prior = Xt_prior.rename(columns = {'X':'X_1'})
    ## Valuation
    # X_T 
    Xt = pd.DataFrame(omega_weights, index = tickers_new, columns=['W'])
    Xt = Xt.merge(prices_new, left_index = True, right_index = True, how = 'left') 
    Xt["X"] = Xt['W'] * Val_Port / Xt['Price']
    Xt = Xt['X'].to_frame()

    # X
    X = Xt.join(Xt_prior, how='outer').fillna(0.0)
    X['Trade'] = X['X'] - X['X_1']

    # Only capitals
    X = X.reset_index().rename( columns = {'index':'Stock'})
    X = X[X['Stock'] != 'Rf'].set_index('Stock') # Posible Redundancia

    
    # Sell 
    to_sell = X['Trade']<0
    if len(X[to_sell]) > 0:
        assets_to_sell = np.ceil((X[to_sell].Trade) * (1+commission))
        cash_gain_per_asset = assets_to_sell * prices_new[to_sell].T * (1-commission)
        #net_cash_gain = cash_gain_per_asset.sum(axis=1)
        operations_sell = assets_to_sell.to_frame().rename(columns = {'Trade':'X'})
        operations_sell['Price'] = prices_new[to_sell].values
        operations_sell['Stock'] = X[to_sell].index.values
        operations_sell['Position'] = cash_gain_per_asset.T.values
        operations_sell['Type'] = 'Sell'
    else:
        #net_cash_gain = 0
        operations_sell = pd.DataFrame(columns=['X','Price','Stock','Position','Type'])
    
    # Buy
    to_buy = X['Trade']>0
    if len(X[to_buy]) > 0:
        assets_to_buy = np.floor((X[to_buy].Trade) * (1-commission))
        cash_invest_per_asset = assets_to_buy * prices_new[to_buy].T * (1+commission)
        #net_cash_invest = cash_invest_per_asset.sum(axis=1)
        operations_buy = assets_to_buy.to_frame().rename(columns = {'Trade':'X'})
        operations_buy['Price'] = prices_new[to_buy].values
        operations_buy['Stock'] = X[to_buy].index.values
        operations_buy['Position'] = cash_invest_per_asset.T.values
        operations_buy['Type'] = 'Buy'
    else:
        #net_cash_invest = 0
        operations_buy = pd.DataFrame(columns=['X','Price','Stock','Position','Type'])

    
    # Buy Risk Free
    current_date_rf_assets = assets_rf[assets_rf['Date'] == fiscal_date]
    if len(current_date_rf_assets) > 0:
        weight_rf = current_date_rf_assets['W'].values[0]
        rf = data[data["fiscalDateEnding"]==fiscal_date].rf.values[0] / 100
        buy_risk_free = Val_Port*weight_rf
        operations_buy_rf = pd.DataFrame([fiscal_date,'Rf',buy_risk_free,rf,buy_risk_free,'Buy'], index = ['Date','Stock','X','Price','Position','Type']).T
        open_pos = pd.DataFrame([fiscal_date,rf,buy_risk_free,'Open'], index = ['Date','Rate','Position','Status']).T
        positions_status_rf = pd.concat([positions_status_rf,open_pos], axis = 0, ignore_index = True)
    else:
         operations_buy_rf = pd.DataFrame(columns = ['Date','Stock','X','Price','Position','Type'])
         buy_risk_free = 0
    # Operations
    operations_new = pd.concat([operations_sell,operations_buy], axis = 0)
    operations_new['Date'] = fiscal_date
    operations_new = operations_new[['Date','Stock','X','Price','Position','Type']]
    operations = pd.concat([operations,operations_new,operations_buy_rf,operations_sell_rf], axis = 0, ignore_index=True)

    # Cash
    #net_cash_remainer = net_cash_gain + income_risk_free - net_cash_invest - buy_risk_free # Redundancia
    #cash = initial_capital - net_cash_remainer # Redundancia
    
    # Portfolio Value
    portfolio_value = pd.DataFrame([fiscal_date,operations[
        (operations['Date'] == fiscal_date) & (operations['Type'] == 'Buy')
        ]['Position'].sum()], index = ['Date','Value_after_commissions']).T
    portfolio_valuation = pd.concat([portfolio_valuation,portfolio_value], axis = 0, ignore_index=True)

    return operations, portfolio_valuation, positions_status_rf, assets_rf

In [ ]:
operations

,Date,Stock,X,Price,Position,Type


In [ ]:
positions_status_rf

,Date,Rate,Position,Status
0,2018-12-31,0.023,1000000.0,Open
1,2019-03-31,0.023,1005750.0,Closed
2,2019-03-31,0.02328,1000000.0,Open
3,2019-06-30,0.02328,1005820.0,Closed
4,2019-09-30,0.02328,1005820.0,Closed
5,2019-09-30,0.0177,933804.664736,Open
6,2019-12-31,0.0177,937936.750377,Closed


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
assets = pd.DataFrame(columns=['Date','Stock','W'])
positions_status_rf = pd.DataFrame(columns=['Date','Rate','Position','Status'])
portfolio_valuation = pd.DataFrame(columns=['Date','Value_after_commissions'])
operations = pd.DataFrame(columns=['Date','Stock','X','Price','Position','Type'])

wo_abc = data[data['Stock'] != 'ABC']

for fiscal_date in data['fiscalDateEnding'].unique()[:5]:
    print(fiscal_date)
    fiscal_date = fiscal_date.strftime('%Y-%m-%d')
    assets_list = pick_assets(data=wo_abc, assets=assets, fiscal_date=fiscal_date)
    omega_weights, assets = omegaAA(data=wo_abc, assets=assets, assets_lists=assets_list, fiscal_date=fiscal_date)
    operations, portfolio_value, positions_status_rf, assets_rf = trade(data=wo_abc, assets=assets, operations=operations, 
                                                             portfolio_valuation=portfolio_valuation, positions_status_rf=positions_status_rf, 
                                                             omega_weights=omega_weights, fiscal_date=fiscal_date)


2018-12-31 00:00:00


AttributeError: 'DataFrame' object has no attribute 'rf'

In [ ]:
assets_rf

,Date,Stock,W
0,2018-12-31,Rf,1.0
1,2019-03-31,Rf,1.0
7,2019-09-30,Rf,1.0
